# Fisher formalism to forecast kSZ SNR.

* ### Compute and store Fisher matrix for the following parameters. 
* ### Check [s2_analyse_fisher_matrix.ipynb](s2_analyse_fisher_matrix.ipynb) to analyse the Fisher matrix. 
    * #### Get the total kSZ SNR.
    * #### Margninalise over radio and CIB+tSZ residuals.
    * #### Marginalise over local-kSZ parameters and obtain reionisation constraints.
* #### Parameters constrained: 6 \$\Lambda{\rm CDM}$ + $\alpha_{\rm radio}$ + $\sigma(\alpha_{\rm radio})$ + $A_{\rm CIB+tSZ}$ + $A_{\rm kSZ}$ + $A_{\rm kSZ}^{h}$ + $\alpha_{\rm kSZ}^{h}$ + $z_{\rm re}^{\rm mid}$ + $\Delta z_{\rm re}$. Parameters are defined here but please see descriptions below for detailed explanations.
    
    * #### $\alpha_{\rm radio}$ and $\sigma(\alpha_{\rm radio})$: Radio residuals.
    * #### $A_{\rm CIB+tSZ}$: Joint CIB and tSZ residuals.
    * #### $A_{\rm kSZ}$: Total kSZ power spectrum amplitude with $D_{\ell}^{\rm kSZ} = A_{\rm kSZ} \times 3 \mu {\rm * K}^2$.
    * #### $A_{\rm kSZ}^{h}$ and $\alpha_{\rm kSZ}^{h}$: Local homogeneous kSZ amplitude and tilt.
    * #### $z_{\rm re}^{\rm mid}$ and $\Delta z_{\rm re}$: Late-time reionisation kSZ mid-point and duration of reionisation (expressed in terms of redshift $z_{\rm re}$).

#### Noise + foreground residuals: tSZ-free x CIB-free cross-ILC from, for example, [ilc_weights_residuals_agora_fg_model.npy](publish/ilc/ilc_weights_residuals_agora_fg_model.npy)

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np, sys, os, scipy as sc, warnings
sys.path.append('modules/')
import tools, ilc, fisher_tools
import matplotlib.cbook
import scipy.ndimage as ndimage
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

#%pylab notebook
%matplotlib inline
from pylab import *

import copy

In [3]:
rcParams['figure.dpi'] = 150
rcParams['figure.facecolor'] = 'white'


# Get the ILC residuals first

In [4]:
#get the ILC residuals first
which_fg_model = 'agora'

fname = 'publish/ilc/ilc_weights_residuals_%s_fg_model.npy' %(which_fg_model)
ilc_dict = np.load(fname, allow_pickle = True).item()
print(ilc_dict.keys())

#get experiment Nl_dict
nl_TP_dict = ilc_dict['nl_TP_dict']

#different types of ILC considered
ilc_keyname_arr = ilc_dict['ilc_keyname_arr']
print(ilc_keyname_arr)

#ILC weights
weights_dict = ilc_dict['weights']
print(weights_dict.keys())

#total ILC residuals
total_ilc_residuals_dict = ilc_dict['total_ilc_residuals']
print(total_ilc_residuals_dict.keys())

if (1): #with CCAT-prime
    fname_withccatp = 'publish/ilc/ilc_weights_residuals_%s_fg_model_withccatp.npy' %(which_fg_model)
    ilc_dict_withccatp = np.load(fname_withccatp, allow_pickle = True).item()

    weights_dict_withccatp = ilc_dict_withccatp['weights']
    
    #get experiment Nl_dict
    nl_TP_dict_withccatp = ilc_dict_withccatp['nl_TP_dict']

    #total ILC residuals
    total_ilc_residuals_dict_withccatp = ilc_dict_withccatp['total_ilc_residuals']
    #print(total_ilc_residuals_dict.keys())
    
    ##print(ilc_dict_withccatp.keys())

#el range
els = ilc_dict['el']

dict_keys(['weights', 'total_ilc_residuals', 'cib_plus_tsz_residuals', 'cib_residuals', 'tsz_residuals', 'noise_residuals', 'radio_residuals', 'nl_TP_dict', 'cl_signal_plus_noise', 'ilc_keyname_arr', 'el'])
['mv' 'tszfree' 'cibfree' 'mvxcibfree' 'mvxtszfree' 'tszfreexcibfree']
dict_keys(['s4_wide', 's4_deep', 'spt3g', 'spt4', 'so_baseline', 'so_goal'])
dict_keys(['s4_wide', 's4_deep', 'spt3g', 'spt4', 'so_baseline', 'so_goal'])


# Experiment names and $f_{\rm sky}$ values

In [5]:
#experiment names and fsky values
exp_specs_dict = {
                  's4_wide': ['S4-Wide', 0.5, 'black'], 
                  #'s4_wide_withccatp': ['S4-Wide + FYST', 0.5], 
                  's4_deep': ['S4-Ultra Deep', 0.03, 'darkred'], 
                  'spt3g': ['SPT-3G', 0.03, 'darkgreen'],
                  'spt4': ['SPT-3G+SPT-4', 0.03, 'tab:green'],
                  'so_baseline': ['SO-Baseline', 0.4, 'royalblue'],
                  'so_baseline_withccatp': ['SO-Baseline + FYST', 0.4, 'tab:blue'], 
                  'so_goal': ['SO-Goal', 0.4, 'darkorange'],
                  'so_goal_withccatp': ['SO-Goal + FYST', 0.4, 'tab:orange'], 
                  }

#ILC keynames
ilc_keyname_dict = {'mv':'MV', 'tszfree': 'tSZ-free', 'cibfree': 'CIB-free'}

# Setting variables for Fisher forecasting
###  <u>Default cosmology:</u> $\texttt{planck_cosmo_version}$ = ${\it Planck}$ 2018.
  * ### Param file used for CAMB: publish/data/params_planck_r_0.0_2018_cosmo.txt
  * ### TT, TE, EE + lowE + lensing in Table 2 of [1807.06209](https://arxiv.org/abs/1807.06209)
  * ### ${\it Planck}$ 2015 is also available.

### <u>Foreground residuals</u>:
  * ### Radio residuals are modelled using two parameters:
      * #### $\alpha_{\rm rad} = -0.76$: Mean spectral index of sources.
      * #### $\sigma(\alpha_{\rm rad}) = 0.2$: Scatter in the spectral index of sources.
  * ### Left over CIB+tSZ residuals are modelled using a single amplitude parameter $A_{\rm CIB+tSZ}=1$.
  
###  <u>kSZ SNR</u>:
* ### A single $A_{\rm kSZ} = 1$ is assumed for the amplitude of the kSZ power spectrum. 
    * ### $D_{\ell}^{\rm kSZ} = A_{\rm kSZ} \times 3 \mu {\rm K}^2$.

###  <u>Local / Reionisation kSZ</u>: We use 4 parameters. 2 for local homogeneous kSZ and 2 for late-time patchy kSZ.
* ### Homogeneous kSZ: Modelled as a power law: $D_{\ell}^{\rm h-kSZ} = A_{\rm kSZ}^{\rm h} \left(\frac{\ell}{\ell_{\rm norm}}\right)^{\alpha_{\rm ksz}^{\rm h}} \mu {\rm K}^2$ where
    * ### $A_{\rm ksz}^{\rm h} = 1.5$ is the amplitude of the homogeneous kSZ power spectrum.
    * ### $\alpha_{\rm ksz}^{\rm h} = 0$ is the tilt.
    * ### $\ell_{\rm norm}=3000$ is the pivot.
* ### Patchy kSZ: Modelled using [AMBER (Chen et al. 2022)](https://arxiv.org/abs/2203.04337) simulations. Parameters used for modelling are
    * ### $z_{\rm re}^{\rm mid} = 7.69$ is the midpoint of reionisation where the fiducial value corresponds to ${\it Planck}$ 2018 results $\tau_{\rm re} = 0.0544$.
    * ### $\Delta z_{\rm re} = 4$ is the duration of reionisation.
    * ### This roughly gives $D_{\ell}^{\rm p-kSZ} \sim 1.5 \mu {\rm K}^2$ and so the total kSZ is roughly $D_{\ell}^{\rm kSZ} = D_{\ell}^{\rm h-kSZ} + D_{\ell}^{\rm p-kSZ} \sim 3 \mu {\rm K}^2$.
        * ### Check [publish/data/cl_patchy_ksz_amber_derivatives.npy](https://github.com/sriniraghunathan/cross_ilc_methods_paper/blob/main/publish/data/cl_patchy_ksz_amber_derivatives.npy).  

In [6]:
#some variables for Fisher forecasting

#expname = 's4_wide'
#fsky = exp_specs_dict[expname][1]
planck_cosmo_version = 2018 ##2015 #2018
camb_folder = 'publish/data/CAMB/planck_%s/' %(planck_cosmo_version)
which_spectra = 'lensed'
max_flux_mJy = 3e-3 #masking threshold used.

#radio-related stuffs
spec_index_radio = -0.76 #Radio spectral index fiducial value
spec_index_radio_step = spec_index_radio/100. #stpe for derivative calculation
spec_index_radio_prior = 0.1 #Radio spectral index prior
alpha_radio = spec_index_radio

spec_index_radio_scatter = 0.2 #Radio spectral index fiducial scatter
spec_index_radio_scatter_step = spec_index_radio_scatter/100.
spec_index_radio_scatter_prior = 0.3 #Radio spectral index scatter prior
alpha_radio_sigma = spec_index_radio_scatter

#CIB+tSZ residuals
Acibtsz = 1.
Acibtsz_step = Acibtsz/100.

In [7]:
#ksz-related stuffs.    
redo_fisher = False

#parameters for total kSZ SNR
dl_ksz_amp_total = 3. #D_{\ell}^{\rm kSZ} = 3 uK^2 
Aksz = 1. #Amplitude of kSZ.
Aksz_step = Aksz / 100. #stpe for derivative calculation

#parameters for homogeneous and reionisation kSZ.
#homogeneous kSZ - similar to Alvarez et al. 2020
Aksz_h = 1.5
alphaksz_h = 0.
Aksz_h_step = Aksz_h/100.
alphaksz_h_step = 0.01

#patchy kSZ part
##zmid = 8.
import tools_for_plotting
import camb
param_dict = tools_for_plotting.get_ini_param_dict()
#print(param_dict)
pars = camb.CAMBparams(max_l_tensor = param_dict['max_l_tensor'], max_eta_k_tensor = param_dict['max_eta_k_tensor'])
pars.set_accuracy(AccuracyBoost = param_dict['AccuracyBoost'], lAccuracyBoost = param_dict['lAccuracyBoost'], lSampleBoost = param_dict['lSampleBoost'],\
    DoLateRadTruncation = param_dict['do_late_rad_truncation'])
pars.set_cosmology(thetastar=param_dict['thetastar'], ombh2=param_dict['ombh2'], omch2=param_dict['omch2'], nnu = param_dict['neff'], mnu=param_dict['mnu'], \
    omk=param_dict['omk'], tau=param_dict['tau'], YHe = param_dict['YHe'], Alens = param_dict['Alens'], \
    num_massive_neutrinos = param_dict['num_nu_massive'])

zmid = camb.get_zre_from_tau(pars,param_dict['tau'])
zdur = 4.
if (0): #precomputed.
    zmid_step = zmid/100.
    zdur_step = zdur/100.

## Pick CMB $C_{\ell}^{\rm xx}$ and derivatives $\dfrac{\partial C_{\ell}^{\rm XX}}{\partial \theta}$ where ${\rm XX} \in [TT, EE, TE]$ and $\theta \in [A_{s},\ h,\ n_{s},\ \Omega_{c}h^{2},\ \Omega_{b}h^{2},\ \tau_{\rm re}]$
### Note:
* $C_{\ell}$ and the derivatives are already stored and also contain additional parameters namely $\sum m_{\nu}$ and $w_{0}$. Both these parameters are fixed during Fisher forecasting.
* Paramters that are added later (below):
  * $A_{\rm kSZ}$ - Amplitude of the kSZ power spectrum $D_{\ell} = 3 \mu {\rm K}^{2}$.
  * $\alpha_{\rm rad}$ - Spectral index of radio point sources.
  * $\sigma(\alpha_{\rm rad})$ - Scatter in the distribution of spectral indices of the radio point sources.
  * $A_{\rm CIB+tSZ}$: Joint CIB and tSZ residuals.
  * $A_{\rm kSZ}$: Total kSZ power spectrum amplitude with $D_{\ell}^{\rm kSZ} = A_{\rm kSZ} \times 3 \mu {\rm * K}^2$.
  * $A_{\rm kSZ}^{h}$ and $\alpha_{\rm kSZ}^{h}$: Local homogeneous kSZ amplitude and tilt.
  * $z_{\rm re}^{\rm mid}$ and $\Delta z_{\rm re}$: Late-time reionisation kSZ mid-point and duration of 

In [8]:
#CMB stuffs
#get fiducual LCDM power spectra computed using CAMB
print('get fiducual LCDM power spectra computed using CAMB')
camb_fname = '%s/cmb_spectra_%s.txt' %(camb_folder, which_spectra)
cl_camb = np.loadtxt(camb_fname)
el_camb = cl_camb[:,0] 
cl_camb_tt = cl_camb[:,1]
dl_fac_camb = el_camb * (el_camb+1)/2/np.pi
cl_dic = {}
cl_dic['TT'] = cl_camb[:,1]
cl_dic['EE'] = cl_camb[:,2]
cl_dic['TE'] = cl_camb[:,3]

#get derivatives of CMB power spectrum for different LCDM parameters.
#They are already computed and stored.
print('get/read derivatives')
camb_deriv_fname = '%s/cmb_spectra_derivs_%s.npy' %(camb_folder, which_spectra)
cl_deriv_dic_tmp = np.load(camb_deriv_fname, allow_pickle = 1).item()
cl_deriv_dic = {}
param_names = []
for p in sorted( cl_deriv_dic_tmp ):
    if p == 'ell': continue    
    cl_deriv_dic[p]={}
    if planck_cosmo_version == 2018:
        cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p]['TT']
        cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p]['EE']
        cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p]['TE']
    else:
        cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p][0]
        cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p][1]
        cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p][2]
    
    if p == 'As' and planck_cosmo_version == 2015:
        cl_deriv_dic[p]['TT'] *= 1e9
        cl_deriv_dic[p]['EE'] *= 1e9
        cl_deriv_dic[p]['TE'] *= 1e9
        
    param_names.append( p )
    
#interpolate CAMB spectra / derivatives on the desired els.
for which_spec in cl_dic:
    cl_dic[which_spec] = np.interp(els, el_camb, cl_dic[which_spec], right=0.)
for p in sorted( param_names ):
    for which_spec in cl_deriv_dic[p]:
        cl_deriv_dic[p][which_spec] = np.interp(els, el_camb, cl_deriv_dic[p][which_spec], right=0.)
        
cl_dic_cmb = copy.deepcopy(cl_dic)

get fiducual LCDM power spectra computed using CAMB
get/read derivatives


## Sanity check - Compare derivatives $\dfrac{\partial C_{\ell}^{\rm XX}}{\partial \theta}$ from ${\it Planck}$ 2015 vs 2018 cosmology.

In [9]:
if (0): #compare 2015 and 2018 derivatives
    planck_cosmo_version_arr = [2015, 2018]
    parent_cl_deriv_dic = {}
    for planck_cosmo_version in planck_cosmo_version_arr:
        camb_deriv_fname = 'publish/data/CAMB/planck_%s/cmb_spectra_derivs_%s.npy' %(planck_cosmo_version, which_spectra)
        cl_deriv_dic_tmp = np.load(camb_deriv_fname, allow_pickle = 1).item()
        cl_deriv_dic = {}
        param_names = []
        for p in sorted( cl_deriv_dic_tmp ):
            if p == 'ell': continue
            cl_deriv_dic[p]={}
            if planck_cosmo_version == 2018:
                cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p]['TT']
                cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p]['EE']
                cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p]['TE']
            else:
                cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p][0]
                cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p][1]
                cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p][2]

            if p == 'As' and planck_cosmo_version == 2015:
                cl_deriv_dic[p]['TT'] *= 1e9
                cl_deriv_dic[p]['EE'] *= 1e9
                cl_deriv_dic[p]['TE'] *= 1e9

            param_names.append( p )
        parent_cl_deriv_dic[planck_cosmo_version] = cl_deriv_dic
        print(param_names)
        
    for p in ['As', 'mnu', 'neff', 'ns', 'ombh2', 'omch2', 'tau', 'thetastar']:
    #for p in ['thetastar']:
        clf()
        ax = subplot(111, yscale = 'log')
        which_spec_dict = {'TT': 'navy', 'EE': 'darkgreen', 'TE': 'darkred'}
        for which_spec in which_spec_dict:
            der2015 = parent_cl_deriv_dic[2015][p][which_spec]
            der2018 = parent_cl_deriv_dic[2018][p][which_spec]
            plot(abs(der2018), color = which_spec_dict[which_spec], ls = '-')
            plot(abs(der2015), color = which_spec_dict[which_spec], ls = '--', lw = 4.)
        show(); sys.exit()
            
        
    

## Get the kSZ temperature power spectrum $C_{\ell}^{\rm kSZ}$ and the derivatives:
* ### Total kSZ SNR: $\dfrac{\partial C_{\ell}^{\rm kSZ}}{\partial A_{\rm kSZ}}$.
* ### Reionisation kSZ: $\dfrac{\partial C_{\ell}^{\rm kSZ}}{\partial \theta}$ where $\theta \in [A_{\rm ksz}^{\rm h}, \alpha_{\rm ksz}^{\rm h}, z_{\rm re}^{\rm mid}, \Delta z_{\rm re}]$.
### Note:
  * ### We assume that the derivatives of $C_{\ell}^{\rm kSZ}$ w.r.t $\Lambda CDM$ parameters is zero. 
      * ### This is not fully correct.
  * ### We also assume that the derivatives of CMB power spectrum $C_{\ell}^{\rm TT}$ w.r.t $A_{\rm kSZ}$ is zero.
  * ### We set EE and TE part of $C_{\ell}^{\rm kSZ} = 0$.


In [10]:
#kSZ stuffs
print('get kSZ spectrum and derivatives')
"""
Assumptions:
1. We assume that the derivatives of cl_kSZ w.r.t LCDM parameters is zero.
    (A) This is not 100 per cent true.
2. We also assume that the derivatives of CMB power spectrum w.r.t AkSZ is zero.
    (A) This is a reasonable assumption.
"""

#get kSZ spectrum now: Flat D_{\ell}^{\rm kSZ} = 3 uK^2 
cl_ksz = fisher_tools.get_cl_ksz(els, Aksz = Aksz, dl_ksz_amp_total = dl_ksz_amp_total)

#add kSZ to cl_dic['TT']
cl_dic['TT'] += cl_ksz

#get deriviatives for Aksz
cl_ksz_low = fisher_tools.get_cl_ksz(els, Aksz = Aksz - Aksz_step, dl_ksz_amp_total = dl_ksz_amp_total)
cl_ksz_high = fisher_tools.get_cl_ksz(els, Aksz = Aksz + Aksz_step, dl_ksz_amp_total = dl_ksz_amp_total)
cl_deriv_dic['Aksz'] = {}
cl_deriv_dic['Aksz']['TT'] = (cl_ksz_high-cl_ksz_low)/( 2 * Aksz_step)
cl_deriv_dic['Aksz']['EE'] = cl_deriv_dic['Aksz']['TT'] * 0.
cl_deriv_dic['Aksz']['TE'] = cl_deriv_dic['Aksz']['TT'] * 0.

#get homogeneous kSZ first
el_norm = 3000.
cl_ksz_homo = fisher_tools.get_homo_ksz(els, Aksz_h = Aksz_h, alphaksz_h = alphaksz_h, el_norm = el_norm)
#add cl_ksz_homo to cl_dic['TT']
cl_dic['TT'] += cl_ksz_homo
#get derivatives
#Aksz_h first
cl_ksz_homo_low = fisher_tools.get_homo_ksz(els, Aksz_h = Aksz_h - Aksz_h_step, alphaksz_h = alphaksz_h, el_norm = el_norm)
cl_ksz_homo_high = fisher_tools.get_homo_ksz(els, Aksz_h = Aksz_h + Aksz_h_step, alphaksz_h = alphaksz_h, el_norm = el_norm)
cl_deriv_dic['Aksz_h'] = {}
cl_deriv_dic['Aksz_h']['TT'] = (cl_ksz_homo_high-cl_ksz_homo_low)/( 2 * Aksz_h_step)
cl_deriv_dic['Aksz_h']['EE'] = cl_deriv_dic['Aksz_h']['TT'] * 0.
cl_deriv_dic['Aksz_h']['TE'] = cl_deriv_dic['Aksz_h']['TT'] * 0.
#alphaksz_h now
cl_ksz_homo_low = fisher_tools.get_homo_ksz(els, Aksz_h = Aksz_h, alphaksz_h = alphaksz_h - alphaksz_h_step, el_norm = el_norm)
cl_ksz_homo_high = fisher_tools.get_homo_ksz(els, Aksz_h = Aksz_h, alphaksz_h = alphaksz_h + alphaksz_h_step, el_norm = el_norm)
cl_deriv_dic['alphaksz_h'] = {}
cl_deriv_dic['alphaksz_h']['TT'] = (cl_ksz_homo_high-cl_ksz_homo_low)/( 2 * alphaksz_h_step)
cl_deriv_dic['alphaksz_h']['EE'] = cl_deriv_dic['alphaksz_h']['TT'] * 0.
cl_deriv_dic['alphaksz_h']['TE'] = cl_deriv_dic['alphaksz_h']['TT'] * 0.

#get patchy kSZ now (from precomputed file.)
patchy_ksz_fname = 'publish/data/cl_patchy_ksz_amber_derivatives.npy'
patchy_ksz_dict = np.load(patchy_ksz_fname, allow_pickle = True).item()
els_, cl_ksz_patchy = patchy_ksz_dict['els'], patchy_ksz_dict['cl_ksz_patchy']
cl_ksz_patchy = np.interp(els, els_, cl_ksz_patchy)
#add cl_ksz_patchy to cl_dic['TT']
cl_dic['TT'] += cl_ksz_patchy

if (0):
    dl_fac = els * (els+1)/2/np.pi
    ax = subplot(111, yscale='log')
    plot(els, dl_fac * cl_dic_cmb['TT'], color = 'gray')
    plot(els, dl_fac * cl_ksz_homo, color = 'orangered')
    plot(els, dl_fac * cl_ksz_patchy, color = 'goldenrod')
    plot(els, dl_fac * cl_dic['TT'], color = 'black')
    xlim(0., 5000.); ylim(0.1, 1e4)
    show(); sys.exit()

#derivatives
cl_ksz_patchy_deriv = patchy_ksz_dict['cl_ksz_patchy_deriv']
for tmp_p in cl_ksz_patchy_deriv:
    curr_cl_ksz_patchy_deriv = cl_ksz_patchy_deriv[tmp_p]
    curr_cl_ksz_patchy_deriv = np.interp(els, els_, curr_cl_ksz_patchy_deriv)
    cl_deriv_dic[tmp_p] = {}
    cl_deriv_dic[tmp_p]['TT'] = curr_cl_ksz_patchy_deriv
    cl_deriv_dic[tmp_p]['EE'] = curr_cl_ksz_patchy_deriv * 0.
    cl_deriv_dic[tmp_p]['TE'] = curr_cl_ksz_patchy_deriv * 0.

#store this in a different dict for future use.
#We will add experiment dependent radio residuals below which will change this dictionary.
cl_dic_ori = copy.deepcopy(cl_dic)
cl_deriv_dic_ori = copy.deepcopy(cl_deriv_dic)

print(cl_deriv_dic_ori.keys())

get kSZ spectrum and derivatives
dict_keys(['As', 'mnu', 'neff', 'ns', 'ombh2', 'omch2', 'tau', 'thetastar', 'ws', 'Aksz', 'Aksz_h', 'alphaksz_h', 'zmid', 'zdur'])


# Define some functions.
#### $\texttt{get_temperature_ilc_residuals}$ - Get the temperature ILC residuals from the ILC dictionary read/stored earlier.
#### $\texttt{get_polarisation_mvilc_residuals}$ - Compute the MV-ILC residuals in polarisation for the desried experiment.
#### $\texttt{get_exp_bands}$ - Get frqeuency bands for the desired experiment.
#### $\texttt{wrapper_get_delta_cl}$ - Wrapper to obtain $\Delta C_{\ell}$.
#### $\texttt{get_radio_residuals}$ - Get ILC radio residuals $C_{\ell}^{\rm rad}$ for the desired experiment.
#### $\texttt{get_radio_spectrum_and_derivatives}$ - Get derivatives of radio power spectrum residuals $\dfrac{\partial C_{\ell}^{\rm rad}}{\partial \theta}$ where $\theta \in [\alpha_{\rm rad}, \sigma(\alpha_{\rm rad})]$.
#### $\texttt{get_cib_tsz_spectrum_and_derivatives}$ - Get derivatives of CIB+tSZ power spectrum residuals $\dfrac{\partial C_{\ell}^{\rm CIB+tSZ}}{\partial A_{\rm cib+tsz}}$.


In [11]:
def get_temperature_ilc_residuals(expname, reqd_ilc_keyname_1, reqd_ilc_keyname_2, quiet = False): #Temperature ILC residuals for Nl
    if not quiet:
        print('\tget ILC residuals for Nl')
    if expname.find('withccatp')>-1:
        dict_to_use = total_ilc_residuals_dict_withccatp
        expname_to_use = expname.replace('_withccatp', '')
    else:
        dict_to_use = total_ilc_residuals_dict
        expname_to_use = expname
    if reqd_ilc_keyname_1 != reqd_ilc_keyname_2:
        reqd_ilc_keyname_12 = '%sx%s' %(reqd_ilc_keyname_1, reqd_ilc_keyname_2)
    else:
        reqd_ilc_keyname_12 = reqd_ilc_keyname_1
    els, total_ilc_residual_mv = dict_to_use[expname_to_use]['mv']
    els, total_ilc_residual_1 = dict_to_use[expname_to_use][reqd_ilc_keyname_1]
    els, total_ilc_residual_2 = dict_to_use[expname_to_use][reqd_ilc_keyname_2]
    els, total_ilc_residual_12 = dict_to_use[expname_to_use][reqd_ilc_keyname_12]
    
    return els, total_ilc_residual_mv, total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12
    
def get_polarisation_mvilc_residuals(expname):
    #get MV-ILC for polarisation
    """
    Note that foregrounds are assumed to be unpolarised here.
    So this should simply be a MV noise estimate after taking beams into account.
    """
    dict_for_ilc = {}
    if expname.find('withccatp')>-1:
        expname_to_use = expname.replace('_withccatp', '')
        dict_for_ilc['EE'] = nl_TP_dict_withccatp[expname_to_use]['P']
    else:
        dict_for_ilc['EE'] = nl_TP_dict[expname]['P']
    bands = get_exp_bands(expname)
    mvilc_pol_residuals, mvilc_pol_weights = ilc.get_mvilc_residual_and_weights(bands, els, dict_for_ilc)
    mvilc_pol_residuals = mvilc_pol_residuals[0]

    if (0):#expname == 's4_wide': #show plot for MV-ILC for pol and compare that will noise.
        clf()
        fsval = 14
        band_color_dict = {95: 'navy', 150: 'darkgreen', 220: 'goldenrod', 285: 'orangered', 345: 'darkred', 
                            '410': 'hotpink', 850: 'black'}
        ax=subplot(111, yscale = 'log')
        noise_arr = []
        for (nu1, nu2) in dict_for_ilc['EE']:
            if nu1 != nu2: continue
            curr_nl = dict_for_ilc['EE'][(nu1, nu2)]

            plot(els, curr_nl, color = band_color_dict[nu1], label = r'%s GHz' %(nu1))
            noise_arr.append(curr_nl)

        #MV-ILC for pol
        plot(els, mvilc_pol_residuals, color = 'black', label = r'MV-ILC')

        if (0): #simple MV noise for pol as a sanity check.
            noise_arr = np.asarray(noise_arr)
            mv_noise_pol = ( np.sum(noise_arr**-2, axis = 0) )**-0.5
            plot(els, mv_noise_pol, color = 'hotpink', lw = 2., ls = '-.', label = r'MV noise for pol.')

        legend(loc = 1, fontsize = fsval - 6, ncol = 5)
        xlabel(r'Multipole $\ell$', fontsize = fsval)
        ylabel(r'Polarisation noise: $C_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval-2)
        xlim(0., 7000.); ylim(1e-7, .01)
        title_str = r'%s: Polarisation noise + MV-ILC' %(exp_specs_dict[expname][0])
        title(title_str, fontsize = fsval)
        show()
        
    return els, mvilc_pol_residuals

def get_exp_bands(expname):
    if expname in ['s4_wide', 's4_deep', 'so_baseline', 'so_goal']:        
        bands = [95, 150, 220, 285]
    elif expname == 'spt3g':
        bands = [95, 150, 220]#, 600, 857]
    elif expname == 'spt4':
        bands = [95, 150, 220, 285, 345]
    elif expname in ['s4_wide_withccatp', 'so_baseline_withccatp', 'so_goal_withccatp']: 
        bands = [95, 150, 220, 285, 345, 410, 850]
        
    return bands

def wrapper_get_delta_cl(els, cl_dic, fsky, 
                            total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12, 
                            total_ilc_residual_mv, mvilc_pol_residuals,
                           ):    
    #get delta_Cl using Knox formula.
    nl11_dic = {}
    nl11_dic['TT'] = total_ilc_residual_1
    nl11_dic['EE'] = mvilc_pol_residuals
    nl11_dic['TE'] = np.copy(total_ilc_residual_1) * 0.

    nl22_dic = {}
    nl22_dic['TT'] = total_ilc_residual_2
    nl22_dic['EE'] = mvilc_pol_residuals
    nl22_dic['TE'] = np.copy(total_ilc_residual_2) * 0.

    nl12_dic = {}
    nl12_dic['TT'] = total_ilc_residual_12
    nl12_dic['EE'] = mvilc_pol_residuals
    nl12_dic['TE'] = np.copy(total_ilc_residual_12) * 0.

    delta_cl_dic = fisher_tools.get_knox_errors_parent(els, cl_dic, nl11_dic, fsky, nl22_dic = nl22_dic, nl12_dic = nl12_dic)
    #print(delta_cl_dic.keys())

    if (1): #MV ILC
        nl_mv_dic = {}
        nl_mv_dic['TT'] = total_ilc_residual_mv
        nl_mv_dic['EE'] = mvilc_pol_residuals
        nl_mv_dic['TE'] = np.copy(total_ilc_residual_mv) * 0.
        delta_cl_dic_mv = fisher_tools.get_knox_errors_parent(els, cl_dic, nl_mv_dic, fsky)
        #print(delta_cl_dic_mv.keys()) 
    
    return delta_cl_dic, delta_cl_dic_mv


#radio stuffs
def get_radio_residuals(expname, which_dnds = 'lagache', 
                        min_flux_mJy=0.1e-3, max_flux_mJy = 3e-3, 
                        spec_index_radio = -0.76, 
                        spec_index_radio_scatter = 0.2,
                        quiet = True): 
    #get radio spectrum now for a given masking threshold.
    if expname.find('withccatp')>-1:
        expname_to_use = expname.replace('_withccatp', '')        
        w1, w2 = weights_dict_withccatp[expname_to_use][reqd_ilc_keyname_1], weights_dict_withccatp[expname_to_use][reqd_ilc_keyname_2] #weights for the two ILC maps.
    else:
        expname_to_use = expname
        w1, w2 = weights_dict[expname_to_use][reqd_ilc_keyname_1], weights_dict[expname_to_use][reqd_ilc_keyname_2] #weights for the two ILC maps.
    bands = get_exp_bands(expname)
    ##print(bands, len(bands), w1.shape, w2.shape); sys.exit()
    radio_cl_dict = tools.get_radio_ilc_residuals(els, bands, w1, w2, [which_dnds], 
                                                  min_flux_mJy = min_flux_mJy, max_flux_mJy = max_flux_mJy, 
                                                  spec_index_radio = spec_index_radio, 
                                                  spec_index_radio_scatter_arr = [spec_index_radio_scatter],
                                                 quiet = quiet)

    res_cl_radio = radio_cl_dict[which_dnds][spec_index_radio_scatter]['res_ilc']
    
    return res_cl_radio
    
def get_radio_spectrum_and_derivatives(expname, cl_dic, cl_deriv_dic, which_dnds = 'lagache', 
                        min_flux_mJy=0.1e-3, max_flux_mJy = 3e-3, 
                        spec_index_radio = -0.76, 
                        spec_index_radio_scatter = 0.2, 
                        quiet = True):
    
    #get radio spectrum
    if not quiet: print('get radio spectrum')
    cl_radio = get_radio_residuals(expname, max_flux_mJy = max_flux_mJy, spec_index_radio = spec_index_radio, spec_index_radio_scatter = spec_index_radio_scatter, quiet=quiet)
    #add radio to cl_dic['TT']
    ##cl_dic['TT'] += cl_radio## - already included in total residuals.

    #get deriviatives for spec_index_radio
    if not quiet: print('get deriviatives for alpha_radio')
    cl_radio_alpha_radio_low = get_radio_residuals(expname, max_flux_mJy = max_flux_mJy, spec_index_radio = spec_index_radio - spec_index_radio_step, spec_index_radio_scatter = spec_index_radio_scatter, quiet=quiet)
    cl_radio_alpha_radio_high = get_radio_residuals(expname, max_flux_mJy = max_flux_mJy, spec_index_radio = spec_index_radio + spec_index_radio_step, spec_index_radio_scatter = spec_index_radio_scatter, quiet=quiet)
    cl_deriv_dic['alpha_radio'] = {}
    cl_deriv_dic['alpha_radio']['TT'] = (cl_radio_alpha_radio_high-cl_radio_alpha_radio_low)/( 2 * spec_index_radio_step)
    cl_deriv_dic['alpha_radio']['EE'] = cl_deriv_dic['alpha_radio']['TT'] * 0.
    cl_deriv_dic['alpha_radio']['TE'] = cl_deriv_dic['alpha_radio']['TT'] * 0.

    #get deriviatives for spec_index_radio_scatter
    if not quiet: print('get deriviatives for alpha_radio_sigma')
    cl_radio_alpha_radio_sigma_low = get_radio_residuals(expname, max_flux_mJy = max_flux_mJy, 
                                                         spec_index_radio = spec_index_radio, 
                                                         spec_index_radio_scatter = spec_index_radio_scatter - spec_index_radio_scatter_step,
                                                        quiet=quiet)
    cl_radio_alpha_radio_sigma_high = get_radio_residuals(expname, max_flux_mJy = max_flux_mJy, 
                                                          spec_index_radio = spec_index_radio, 
                                                          spec_index_radio_scatter = spec_index_radio_scatter + spec_index_radio_scatter_step,
                                                         quiet=quiet)
    cl_deriv_dic['alpha_radio_sigma'] = {}
    cl_deriv_dic['alpha_radio_sigma']['TT'] = (cl_radio_alpha_radio_sigma_high-cl_radio_alpha_radio_sigma_low)/( 2 * spec_index_radio_scatter_step)
    cl_deriv_dic['alpha_radio_sigma']['EE'] = cl_deriv_dic['alpha_radio_sigma']['TT'] * 0.
    cl_deriv_dic['alpha_radio_sigma']['TE'] = cl_deriv_dic['alpha_radio_sigma']['TT'] * 0.
    
    return cl_radio, cl_dic, cl_deriv_dic

    
#CIB+tSZ residuals
def get_cib_tsz_spectrum_and_derivatives(expname, cl_dic, cl_deriv_dic, reqd_ilc_keyname_1, reqd_ilc_keyname_2 = None, Acibtsz = 1., Acibtsz_step = None):
    if reqd_ilc_keyname_1 != reqd_ilc_keyname_2:
        reqd_ilc_keyname_str = '%sx%s' %(reqd_ilc_keyname_1, reqd_ilc_keyname_2)
    else:
        reqd_ilc_keyname_str = reqd_ilc_keyname_1
    cl_cibtsz_ori = ilc_dict['cib_plus_tsz_residuals']['s4_wide'][reqd_ilc_keyname_str]
    cl_cibtsz = Acibtsz * np.copy( cl_cibtsz_ori )
    #add CIB+tSZ to cl_dic['TT']
    ##cl_dic['TT'] += cl_cibtsz## - already included in total residuals.
    
    #now get derivatives
    if Acibtsz_step == None: Acibtsz_step = Acibtsz / 100.
        
    cl_cibtsz_low = np.copy( cl_cibtsz_ori ) * ( Acibtsz - Acibtsz_step)
    cl_cibtsz_high = np.copy( cl_cibtsz_ori ) * ( Acibtsz + Acibtsz_step)
    cl_deriv_dic['Acibtsz'] = {}
    cl_deriv_dic['Acibtsz']['TT'] = (cl_cibtsz_high-cl_cibtsz_low)/( 2 * Acibtsz_step)
    cl_deriv_dic['Acibtsz']['EE'] = cl_deriv_dic['Acibtsz']['TT'] * 0.
    cl_deriv_dic['Acibtsz']['TE'] = cl_deriv_dic['Acibtsz']['TT'] * 0.
        
    return cl_cibtsz, cl_dic, cl_deriv_dic


#check derivatives
def plot_and_check_derivatives(cl_deriv_dic, reqd_param_arr = None):
    which_spec_dict = {'TT': 'navy'}#, 'EE': 'darkgreen', 'TE': 'darkred'}
    clf()
    ax = subplot(111, yscale = 'log')
    for p in list(cl_deriv_dic.keys()):
        if reqd_param_arr is not None:
            if p not in reqd_param_arr: continue
        print(p, reqd_param_arr)
        for which_spec in which_spec_dict:
            labval = r'%s' %(tools_for_plotting.get_latex_param_str(p))
            print(cl_deriv_dic[p][which_spec])
            plot(cl_deriv_dic[p][which_spec], ls = '-', label = labval)#, color = which_spec_dict[which_spec])
            plot(-cl_deriv_dic[p][which_spec], ls = '-.')#, color = which_spec_dict[which_spec])
    show(); 
    #sys.exit()
    
#plot_and_check_derivatives(cl_deriv_dic, reqd_param_arr = ['alpha_radio', 'alpha_radio_sigma'])

In [12]:
#make sure there are no infs and nans in derivatives
for p in cl_deriv_dic:
    for which_spec in cl_deriv_dic[p]:
        cl_deriv_dic[p][which_spec][np.isnan(cl_deriv_dic[p][which_spec])] = 0.
        cl_deriv_dic[p][which_spec][np.isinf(cl_deriv_dic[p][which_spec])] = 0.

if (0):
    #plot_and_check_derivatives(cl_deriv_dic, reqd_param_arr=['zmid', 'zdur'])
    #plot_and_check_derivatives(cl_deriv_dic, reqd_param_arr=['zdur'])
    plot_and_check_derivatives(cl_deriv_dic, reqd_param_arr=['Acibtsz'])


# Run Fisher forecasts for different experiments.
* ### Power spectra used: TT/EE/TE.
  * #### Multipole $\ell$ ranges:
   * #### EE/TE: $\ell \in [30, 5000]$.
   * #### TT:
     * #### CMB and radio: $\ell \in [30, \ell_{\rm max}^{TT}]$ with $\ell_{\rm max}^{TT} \in [5000, 6000, 7000]$.
     * #### $\ell_{\rm max}^{\rm TT, kSZ}$ for kSZ = [3000, 3500, 4000, 4500, 5000] for both MV and Cross-ILC.
* ### Experiments: 
  * #### SPT-3G, SPT-3G + SPT-4, SO-Baseline, SO-Baseline+FYST, SO-Goal, SO-Goal+FYST, S4-Wide and S4-Ultra Deep.
* ### Priors: Will be added later in [s2_analyse_fisher_matrix.ipynb](s2_analyse_fisher_matrix.ipynb).
* ### Parameters constrained and fixed: Will be added later in [s2_analyse_fisher_matrix.ipynb](s2_analyse_fisher_matrix.ipynb).
  * #### $\theta \in [A_{s},\ h,\ n_{s},\ \Omega_{c}h^{2},\ \Omega_{b}h^{2},\ \tau_{\rm re},  A_{\rm CIB+tSZ}, \alpha_{\rm rad}, \sigma(\alpha_{\rm rad}), A_{\rm kSZ}, A_{\rm ksz}^{\rm h}, \alpha_{\rm ksz}^{\rm h}, z_{\rm re}^{\rm mid}, \Delta z_{\rm re}]$.
  * #### Total kSZ SNR: We will fix the following parameters in [s2_analyse_fisher_matrix.ipynb](s2_analyse_fisher_matrix.ipynb): $\theta \in [A_{\rm ksz}^{\rm h}, \alpha_{\rm ksz}^{\rm h}, z_{\rm re}^{\rm mid}, \Delta z_{\rm re}]$. 
  * #### Reionisation kSZ: We will fix the following parameters in [s2_analyse_fisher_matrix.ipynb](s2_analyse_fisher_matrix.ipynb): $[A_{\rm ksz}$.


In [24]:
#parameters for constraining/fixing, power spectra to be used, \ell ranges and priors.
pspectra_to_use = ['TT', 'EE', 'TE']
pspectra_to_use_str = ''.join(pspectra_to_use)
min_l_pol, max_l_pol = 30, 5000 
#min_l_temp, max_l_temp = 30, 5000 #we will set different ell ranges for kSZ. Check max_l_temp_for_ksz_arr below.
min_l_temp = 30 #we will set different ell ranges for kSZ. Check max_l_temp_for_ksz_arr below.
max_l_temp_arr = [5000, 6000, 7000]
prior_dic = {'tau':0.007, #Planck tau prior
             'alpha_radio': spec_index_radio_prior, 
             'alpha_radio_sigma': spec_index_radio_scatter_prior,
             'Aksz_h': 0.1,
             'alphaksz_h': 0.1,
             #'Acibtsz': 0.0001
                }#, 'Akszh': dl_ksz_amp_homo/10.}#02}#02} 

expname_arr = ['spt3g', 'spt4', 'so_baseline', 'so_baseline_withccatp', 'so_goal', 'so_goal_withccatp', 's4_wide', 's4_deep']
#expname_arr = ['so_baseline', 'so_baseline_withccatp', 'so_goal', 'so_goal_withccatp']
#expname_arr = ['s4_wide', 'spt4']


if (1):
    pspectra_to_use = ['TT']
    pspectra_to_use_str = ''.join(pspectra_to_use)
    max_l_temp_arr = [5000]
    expname_arr = ['s4_wide']

#output file name
fisher_opfname = 'publish/fisher/F_mat_lcdm_foregrounds_allkSZ_%s_%gmJymask.npy' %(pspectra_to_use_str, max_flux_mJy*1e3)
print(fisher_opfname); ##sys.exit()

#for printing kSZ SNR
desired_param = 'Aksz'

if os.path.exists(fisher_opfname) and not redo_fisher:
    #print('\n\nAlready complete. Check %s\n\n' %(fisher_opfname))
    parent_F_dic = np.load(fisher_opfname, allow_pickle=True).item()
else: 
    parent_F_dic = {}

quiet = True
for max_l_temp in max_l_temp_arr:
    print('lmax_TT = %s' %(max_l_temp))
    maxltempkeyname = 'lmax_%s' %(max_l_temp)
    if maxltempkeyname in parent_F_dic:
        print('\t\tAlready complete for %s' %(maxltempkeyname))
        continue
    parent_F_dic[maxltempkeyname] = {}
    for expname in expname_arr:
        print('\n\tExperiment = %s' %(expname))
        parent_F_dic[maxltempkeyname][expname] = {}

        ilcname_arr = ['MV', 'cross-ILC']
        latex_str = ''
        #for curr_delta_cl_dic, ilcname in zip(delta_cl_dic_arr, ilcname_arr):
        for ilcname in ilcname_arr:
            
            if ilcname == 'cross-ILC':
                reqd_ilc_keyname_1 = 'tszfree'
                reqd_ilc_keyname_2 = 'cibfree'
            else:
                reqd_ilc_keyname_1 = reqd_ilc_keyname_2 = 'mv'

            if reqd_ilc_keyname_1 != reqd_ilc_keyname_2:
                reqd_ilc_keyname_12 = '%sx%s' %(reqd_ilc_keyname_1, reqd_ilc_keyname_2)
            else:
                reqd_ilc_keyname_12 = reqd_ilc_keyname_1
            
            parent_F_dic[maxltempkeyname][expname][ilcname] = {}
        
            cl_dic_forthisexp = copy.deepcopy(cl_dic_ori)
            cl_deriv_dic_forthisexp = copy.deepcopy(cl_deriv_dic_ori)

            #print(reqd_ilc_keyname_1, reqd_ilc_keyname_2); sys.exit()
            #get experiment-dependent radio stuffs
            cl_radio, cl_dic_forthisexp, cl_deriv_dic_forthisexp = get_radio_spectrum_and_derivatives(expname, cl_dic_forthisexp, cl_deriv_dic_forthisexp, max_flux_mJy = max_flux_mJy, 
                                               spec_index_radio = spec_index_radio, 
                                               spec_index_radio_scatter = spec_index_radio_scatter)

            #get experiment-dependent CIB+tSZ stuffs
            cl_cib_tsz, cl_dic_forthisexp, cl_deriv_dic_forthisexp = get_cib_tsz_spectrum_and_derivatives(expname, cl_dic_forthisexp, cl_deriv_dic_forthisexp, 
                                                                                                          reqd_ilc_keyname_1, reqd_ilc_keyname_2 = reqd_ilc_keyname_2, 
                                                                                                          Acibtsz = Acibtsz, Acibtsz_step = Acibtsz_step)

            param_names = list( cl_deriv_dic_forthisexp.keys() )
            ###print(param_names); sys.exit()

            cl_dic = copy.deepcopy(cl_dic_forthisexp)
            cl_deriv_dic = copy.deepcopy(cl_deriv_dic_forthisexp)
            
            ##print(reqd_ilc_keyname_1, reqd_ilc_keyname_2, reqd_ilc_keyname_12); sys.exit()

            if (0):
                clf()
                fsval = 14
                ax = subplot(111, yscale = 'log')
                dl_fac = els * (els+1)/2/np.pi
                which_spec = 'TT'
                plot(els, dl_fac * cl_dic_cmb[which_spec], color = 'gray', label = r'CMB: TT')
                plot(els, dl_fac * cl_ksz, color = 'purple', label = r'kSZ')
                plot(els, dl_fac * cl_cib_tsz, color = 'red', label = r'CIB+tSZ')
                plot(els, dl_fac * cl_cib_tsz, color = 'red', ls = '-.')
                plot(els, dl_fac * cl_radio, color = 'royalblue', label = r'Radio: %s' %(exp_specs_dict[expname][0]))
                plot(els, dl_fac * -cl_radio, color = 'royalblue', ls = '-.')
                if (1):
                    spec_index_radio_arr = [spec_index_radio-0.2, spec_index_radio-0.1, spec_index_radio+0.1, spec_index_radio+0.2]
                    for curr_spec_index_radio in spec_index_radio_arr:
                        cl_radio_tmp = get_radio_residuals(expname, spec_index_radio = curr_spec_index_radio, quiet=True)
                        plot(els, dl_fac * -cl_radio_tmp, ls = '-.', lw=0.5, label=r'%.3f' %(curr_spec_index_radio))
                if (1):
                    spec_index_radio_scatter_arr = [0.01, 0.6]
                    for curr_spec_index_radio_scatter in spec_index_radio_scatter_arr:
                        cl_radio_tmp = get_radio_residuals(expname, spec_index_radio_scatter = curr_spec_index_radio_scatter, quiet=True)
                        plot(els, dl_fac * -cl_radio_tmp, color='black', ls = '-', lw = 4., alpha = 0.1)

                plot([],[], 'k-.', label = r'Negative')
                xlabel(r'Multipole $\ell$', fontsize = fsval); ylabel(r'$D_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval)
                xlim(0., 5000.); ylim(0.1, 1e4)
                legend(loc = 2, fontsize = fsval-4, ncol=2)
                show(); sys.exit()

            bands = get_exp_bands(expname)

            #get MV polarisation ILC residuals
            els, mvilc_pol_residuals = get_polarisation_mvilc_residuals(expname)
            dl_fac = els * (els+1)/2/np.pi

            #get temperature ILC residuals
            els, total_ilc_residual_mv, total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12 = get_temperature_ilc_residuals(expname, reqd_ilc_keyname_1, reqd_ilc_keyname_2, quiet=quiet)
            '''
            if (0): #show ILC residuals, CMB, and kSZ spectra
                clf()
                fsval = 14
                ax = subplot(111, yscale = 'log')
                plot(els, dl_fac * cl_dic['TT'], label = r'CMB: TT', lw = 2., color = 'gray')
                plot(els, dl_fac * cl_ksz, label = r'kSZ', lw = 2., color = 'purple')
                plot(els, dl_fac * total_ilc_residual_mv, label = r'ILC: MV', color = 'darkgreen')
                plot(els, dl_fac * total_ilc_residual_1, label = r'ILC: %s' %(ilc_keyname_dict[reqd_ilc_keyname_1]), color = 'goldenrod')
                plot(els, dl_fac * total_ilc_residual_2, label = r'ILC: %s' %(ilc_keyname_dict[reqd_ilc_keyname_2]), color = 'orangered')
                if (1): #cross-ILC
                    plot(els, dl_fac * total_ilc_residual_12, lw = 1.5, label = r'ILC: %s x %s' %(ilc_keyname_dict[reqd_ilc_keyname_1], ilc_keyname_dict[reqd_ilc_keyname_2]), color = 'darkred')
                    plot(els, dl_fac * -total_ilc_residual_12, lw = 1.5, alpha = 0.3, color = 'darkred', label = r'ILC: %s x %s (Negative)' %(ilc_keyname_dict[reqd_ilc_keyname_1], ilc_keyname_dict[reqd_ilc_keyname_2]))

                xlabel(r'Multipole $\ell$', fontsize = fsval)
                ylabel(r'$D_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval)
                xlim(0., 5000.); ylim(1., 1e4)
                legend(loc = 1, fontsize = fsval-4)
                title(r'%s' %(exp_specs_dict[expname][0]), fontsize = fsval)
                show(); 
                continue
            '''

            #get delta_cl
            expname_str, fsky, colorval = exp_specs_dict[expname]
            delta_cl_dic, delta_cl_dic_mv = wrapper_get_delta_cl(els, cl_dic, fsky, 
                                    total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12, 
                                    total_ilc_residual_mv, mvilc_pol_residuals,
                                   )

            '''
            if expname == 's4_wide': #show CMB and kSZ spectra; and delta_cl for ILC.
                clf()
                fsval = 14
                ax = subplot(111, yscale = 'log')
                plot(els, dl_fac * cl_dic['TT'], label = r'CMB: TT', lw = 2., color = 'gray')
                plot(els, dl_fac * cl_ksz, label = r'kSZ', lw = 2., color = 'purple')

                delta_cl = delta_cl_dic['TT']
                errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * delta_cl, color = 'black', label = 'Cross-ILC')

                delta_cl_mv = delta_cl_dic_mv['TT']
                errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * delta_cl_mv, color = 'lightgreen', label = 'MV')

                xlabel(r'Multipole $\ell$', fontsize = fsval)
                ylabel(r'$D_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval)
                xlim(0., 5000.); ylim(1., 1e4)
                legend(loc = 1, fontsize = fsval-4)
                title(r'%s: Temperature' %(exp_specs_dict[expname][0]), fontsize = fsval)
                show(); ##sys.exit()
            '''

            #get Fisher / COV matrices
            if not quiet:
                print('\t\tObtain Fisher matrix and parameter constraints.')
            delta_cl_dic_arr = [delta_cl_dic_mv, delta_cl_dic]
            print ('\n')
            if (0):##curr_delta_cl_dic_str == 'MV':
                max_l_temp_for_ksz_arr = [3000]#, 3000]
            else:
                max_l_temp_for_ksz_arr = [3000, 3500, 4000, 4500, 5000]
                
                
            curr_delta_cl_dic = delta_cl_dic

            for max_l_temp_for_ksz in max_l_temp_for_ksz_arr:        

                #set kSZ derivatives to zero beyond the desired \ell range for kSZ
                cl_deriv_dic = copy.deepcopy(cl_deriv_dic_forthisexp)
                cl_deriv_dic['Aksz']['TT'][max_l_temp_for_ksz:] = 0.
                cl_deriv_dic['Aksz_h']['TT'][max_l_temp_for_ksz:] = 0.
                cl_deriv_dic['alphaksz_h']['TT'][max_l_temp_for_ksz:] = 0.
                cl_deriv_dic['zmid']['TT'][max_l_temp_for_ksz:] = 0.
                cl_deriv_dic['zdur']['TT'][max_l_temp_for_ksz:] = 0.

                #get Fisher matrix.
                F_mat = fisher_tools.get_fisher_matrix(els, cl_deriv_dic, curr_delta_cl_dic, param_names, pspectra_to_use = pspectra_to_use,\
                            min_l_temp = min_l_temp, max_l_temp = max_l_temp, min_l_pol = min_l_pol, max_l_pol = max_l_pol)
                ##print('\t\tFisher matrix obtained.')

                fisher_keyname = (max_l_temp_for_ksz, max_l_temp)
                parent_F_dic[maxltempkeyname][expname][ilcname][fisher_keyname] = [np.copy(F_mat), np.copy(param_names)]

                #just print kSZ SNR
                if (1):###not constrain_reion: 

                    fix_params = ['mnu', 'ws', 'neff', 'alpha_radio', 'alpha_radio_sigma', 'zmid', 'zdur', 'Aksz_h', 'alphaksz_h']#, 'Acibtsz'])

                    F_mat_mod = np.copy(F_mat)
                    param_names_mod = np.copy(param_names)

                    #add priors.
                    F_mat_mod = fisher_tools.add_priors(F_mat_mod, param_names_mod, prior_dic)
                    ##print('\t\tPriors set.')

                    #fix desired parameters.
                    F_mat_mod, param_names_mod = fisher_tools.fix_params(F_mat_mod, param_names_mod, fix_params)
                    #print('\t\tUndesired parameters (%s) fixed.' %(fix_params))

                    cov_mat = np.linalg.inv(F_mat_mod)
                    param_names_mod = np.asarray(param_names_mod)
                    pind = np.where(param_names_mod == desired_param)[0][0]
                    pcntr1, pcntr2 = pind, pind
                    cov_inds_to_extract = [(pcntr1, pcntr1), (pcntr1, pcntr2), (pcntr2, pcntr1), (pcntr2, pcntr2)]
                    cov_extract = np.asarray( [cov_mat[ii] for ii in cov_inds_to_extract] ).reshape((2,2))
                    sigma_Aksz = cov_extract[0,0]**0.5

                    snr_Aksz = Aksz/sigma_Aksz
                    print( '\t\tILC = %s; lmax for kSZ = %s; AkSZ SNR = %.2f' %(ilcname, max_l_temp_for_ksz, snr_Aksz) )
                    latex_str = '%s & %.2f' %(latex_str, snr_Aksz); #sys.exit()

        if (1):##not constrain_reion:
            latex_str = latex_str.strip(' & ')
            print('\t\t\t', latex_str); ##sys.exit()
    np.save(fisher_opfname, parent_F_dic)
    print('\tpushing Fisher matrix for %s to %s\n\n' %(maxltempkeyname, fisher_opfname))
    
print('\n\nFisher matrix dumped in %s\n\n' %(fisher_opfname))


publish/fisher/F_mat_lcdm_foregrounds_allkSZ_TT_3mJymask.npy
lmax_TT = 5000

	Experiment = s4_wide


		ILC = MV; lmax for kSZ = 3000; AkSZ SNR = 30.77
		ILC = MV; lmax for kSZ = 3500; AkSZ SNR = 54.54
		ILC = MV; lmax for kSZ = 4000; AkSZ SNR = 82.92
		ILC = MV; lmax for kSZ = 4500; AkSZ SNR = 93.79
		ILC = MV; lmax for kSZ = 5000; AkSZ SNR = 6.77


		ILC = cross-ILC; lmax for kSZ = 3000; AkSZ SNR = 24.50
		ILC = cross-ILC; lmax for kSZ = 3500; AkSZ SNR = 36.39
		ILC = cross-ILC; lmax for kSZ = 4000; AkSZ SNR = 46.61
		ILC = cross-ILC; lmax for kSZ = 4500; AkSZ SNR = 50.78
		ILC = cross-ILC; lmax for kSZ = 5000; AkSZ SNR = 53.17
			 30.77 & 54.54 & 82.92 & 93.79 & 6.77 & 24.50 & 36.39 & 46.61 & 50.78 & 53.17
	pushing Fisher matrix for lmax_5000 to publish/fisher/F_mat_lcdm_foregrounds_allkSZ_TT_3mJymask.npy




Fisher matrix dumped in publish/fisher/F_mat_lcdm_foregrounds_allkSZ_TT_3mJymask.npy


